In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import warnings
import re
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.float_format', '{:,.4f}'.format)
pd.set_option('display.max_rows', 80)

df = pd.read_csv("car_age_0_updated_dataset.csv", low_memory=False)
print(f"Initial shape: {df.shape}")

counts = df['od_claim_count'].value_counts().sort_index()
print(counts)

df['policy_start_date'] = pd.to_datetime(df['policy_start_date'])
start_date = df['policy_start_date'].min()
end_date = df['policy_start_date'].max()

print(f"Start date: {start_date}")
print(f"End date:   {end_date}")

print("\n Data Types")
with pd.option_context('display.max_rows', None):
    df.info()

print("\n Top Missing Value Columns")
missing_percent = (df.isnull().sum() / len(df)) * 100
print(missing_percent.sort_values(ascending=False).head(15))

print("\n Numerical Feature Summary")
with pd.option_context('display.max_rows', None, 'display.float_format', '{:,.2f}'.format):
    print(df.describe(include='number').transpose())

Initial shape: (82994, 57)
od_claim_count
0.0000    61784
1.0000    15169
2.0000     4498
3.0000     1092
4.0000      315
5.0000       85
6.0000       31
7.0000       11
8.0000        6
9.0000        3
Name: count, dtype: int64
Start date: 2018-09-10 00:00:00
End date:   2025-09-30 00:00:00

 Data Types
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82994 entries, 0 to 82993
Data columns (total 57 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Unnamed: 0                  82994 non-null  int64         
 1   base_policy                 82994 non-null  object        
 2   od_claim_count              82994 non-null  float64       
 3   car_age                     82994 non-null  float64       
 4   experian_rank_final         77375 non-null  float64       
 5   quarterly_service           37 non-null     float64       
 6   pps_mapped                  81321 non-null  float64       
 7   norma

In [2]:
# 1. Preprocessing + Feature Engineering

# 1a. Drop columns
missing_percent = (df.isnull().sum() / len(df)) * 100
cols_mostly_empty = missing_percent[missing_percent > 70].index.tolist()
unique_counts = df.nunique()
cols_single_value = unique_counts[unique_counts == 1].index.tolist()

cols_to_drop = list(set(cols_mostly_empty + cols_single_value))

cols_to_drop.extend([
    'Unnamed: 0', 
    'policy_start_period', 
    'policy_start_year', 
    'user_agent', 
    'device_vendor_category',
    'base_policy',           
    'recommended_idv',
    'previous_policy_expired',
    'policy_start_month',
    'policy_created_on',
    'cc',
    'model',
    'make',
    'ex_showroom_price',
    'customer_age',
    'exposure_calculated', 
    'variant_bracket',
    'customer_age_group_ordinal',
    'is_rsa',
    'intermediary_id', 
    'Status2'
])

key_cols = ['od_claim_count', 'exposure', 'policy_start_date']
cols_to_drop = [col for col in cols_to_drop if col not in key_cols]

df_cleaned = df.drop(columns=cols_to_drop)
print(f"Shape after dropping columns: {df_cleaned.shape}")

# 1b. Sanitize Feature Names
original_cols = df_cleaned.columns.tolist()
sanitized_cols = [re.sub(r'[^A-Za-z0-9_]+', '_', col) for col in original_cols]
df_cleaned.columns = sanitized_cols

renamed_cols_dict = {orig: new for orig, new in zip(original_cols, sanitized_cols) if orig != new}
if renamed_cols_dict:
    print("Renamed columns:")
    for orig, new in renamed_cols_dict.items():
        print(f"  '{orig}'  =>  '{new}'")

# 1c. Handle date Column
DATE_COL_NAME = 'policy_start_date'
df_cleaned[DATE_COL_NAME] = pd.to_datetime(df_cleaned[DATE_COL_NAME])

# 1d. Define Feature, Target, and Weight
TARGET = 'od_claim_count'
WEIGHT = 'exposure'
DATE_COL = 'policy_start_date'
features = [col for col in df_cleaned.columns if col not in [TARGET, WEIGHT, DATE_COL]]

# 1e. Handle Categorical Features & NaNs
categorical_features = df_cleaned[features].select_dtypes(include=['object', 'category']).columns.tolist()

for col in categorical_features:
    df_cleaned[col] = df_cleaned[col].astype('category')

# 1f. Print Feature Lists
print(f"\nFinal Features for Model Training ({len(features)})")
numerical_features = df_cleaned[features].select_dtypes(include=np.number).columns.tolist()
print(f"\nNumerical Features ({len(numerical_features)}):")
for col in numerical_features:
    print(f"  - {col}")

print(f"\nCategorical Features ({len(categorical_features)}):")
for col in categorical_features:
    print(f"  - {col}")

# 1g. Feature Cardinality
print("\n Feature Cardinality")

cardinality = df_cleaned[features].nunique().sort_values(ascending=False)

print("Features with the most unique values:")
with pd.option_context('display.max_rows', 30):
    print(cardinality.head(30))

# Identify high-cardinality categorical features
high_card_cols = [
    col for col in categorical_features 
    if df_cleaned[col].nunique() > 50
]

Shape after dropping columns: (82994, 19)
Renamed columns:
  'Product+Plan'  =>  'Product_Plan'

Final Features for Model Training (16)

Numerical Features (6):
  - experian_rank_final
  - variant_bracket_mapped
  - cc_group_ordinal
  - hit_flag_service
  - personal_loan_flag
  - embedded_red_flag

Categorical Features (10):
  - corrected_body_type
  - fuel_type
  - transmission_type
  - city_mapped
  - Product_Type2
  - make_mapped
  - cc_group
  - customer_age_group
  - Product_Plan
  - recommended_idv_grouped

 Feature Cardinality
Features with the most unique values:
make_mapped                19
city_mapped                11
experian_rank_final         6
corrected_body_type         6
recommended_idv_grouped     6
cc_group_ordinal            4
fuel_type                   4
cc_group                    4
customer_age_group          4
variant_bracket_mapped      3
hit_flag_service            2
personal_loan_flag          2
embedded_red_flag           2
transmission_type           2
Pr

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

print("Filtering data from Jan 2022 to Apr 2025...")
analysis_start_date = '2022-01-01'
analysis_end_date = '2025-04-30'

df_cleaned['policy_start_date'] = pd.to_datetime(df_cleaned['policy_start_date'])

df_analysis_filtered = df_cleaned[
    (df_cleaned['policy_start_date'] >= analysis_start_date) &
    (df_cleaned['policy_start_date'] <= analysis_end_date)
].copy()

print(f"New shape: {df_analysis_filtered.shape}")
df_time = df_analysis_filtered.set_index('policy_start_date')

# Baseline Shift (P(y))
print("\n Baseline Shift (Overall Risk)")
q_baseline = df_time.resample('Q').agg(
actual_count=('od_claim_count', 'sum'),
exposure=('exposure', 'sum')
)
q_baseline = q_baseline[q_baseline['exposure'] > 0]
q_baseline['Actual IR'] = q_baseline['actual_count'] / q_baseline['exposure']

print(q_baseline[['Actual IR']])

plt.figure(figsize=(12, 6))
q_baseline['Actual IR'].plot(marker='o', linestyle='-')
plt.title('Baseline Shift (Overall Actual IR by Quarter)', fontsize=16)
plt.ylabel('Actual Incidence Rate (IR)')
plt.xlabel('Quarter')
plt.show()

In [4]:
# 2. Time-Based Data Splitting

TRAIN_START = '2022-01-01'
TRAIN_END = '2024-08-31'
TEST_START = '2024-09-01'
TEST_END = '2024-12-31'
OOT_START = '2025-01-01'
OOT_END = '2025-03-31'

# Create the sets
train_df = df_cleaned[
    (df_cleaned[DATE_COL] >= TRAIN_START) &
    (df_cleaned[DATE_COL] <= TRAIN_END)
].copy()

test_df = df_cleaned[
    (df_cleaned[DATE_COL] >= TEST_START) &
    (df_cleaned[DATE_COL] <= TEST_END)
].copy()

oot_df = df_cleaned[
    (df_cleaned[DATE_COL] >= OOT_START) &
    (df_cleaned[DATE_COL] <= OOT_END)
].copy()

print(f"  Train set shape: {train_df.shape} (Dates: {train_df[DATE_COL].min().date()} to {train_df[DATE_COL].max().date()})")
print(f"  Test set shape: {test_df.shape} (Dates: {test_df[DATE_COL].min().date()} to {test_df[DATE_COL].max().date()})")
print(f"  OOT set shape:   {oot_df.shape} (Dates: {oot_df[DATE_COL].min().date()} to {oot_df[DATE_COL].max().date()})")

  Train set shape: (50031, 19) (Dates: 2022-01-01 to 2024-08-31)
  Test set shape: (8614, 19) (Dates: 2024-09-01 to 2024-12-31)
  OOT set shape:   (5809, 19) (Dates: 2025-01-01 to 2025-03-31)


In [6]:
# 3. Model Training (LightGBM Poisson Regressor)

import optuna
import lightgbm as lgb
from sklearn.metrics import mean_poisson_deviance

# Prepare data splits
X_train, y_train, w_train = train_df[features], train_df[TARGET], train_df[WEIGHT]
X_test, y_test, w_test = test_df[features], test_df[TARGET], test_df[WEIGHT]
X_oot, y_oot, w_oot = oot_df[features], oot_df[TARGET], oot_df[WEIGHT]

# Define the Optuna objective function
def objective(trial):
    params = {
        'objective': 'poisson',
        'metric': 'poisson',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'n_jobs': -1,
        'seed': 42,
        'n_estimators': 1000, 
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 60),
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 50, 200),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0, log=True),
    }

    # Train model
    model = lgb.LGBMRegressor(**params)
    
    model.fit(
        X_train, y_train,
        sample_weight=w_train,
        eval_set=[(X_test, y_test)],
        eval_sample_weight=[w_test],
        eval_metric='poisson',
        callbacks=[lgb.early_stopping(150, verbose=False)],
        categorical_feature=categorical_features
    )
    
    
    return model.best_score_['valid_0']['poisson']

# Create and run Optuna study - want to minimize poisson deviance
study = optuna.create_study(direction='minimize')

# Start optimization. 
study.optimize(objective, n_trials=50, show_progress_bar=True)

# Get best parameters
print("\nOptuna study complete.")
print(f"Best trial (poisson): {study.best_value:.6f}")
print("Best parameters found:")
print(study.best_params)

# Train model with best parameters
best_params = study.best_params.copy()

final_lgb_params = {
    'objective': 'poisson',
    'metric': 'poisson',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'n_jobs': -1,
    'seed': 42,
    'n_estimators': 2000,
    **best_params 
}

print("\nTraining Final LightGBM model with best parameters...")
model = lgb.LGBMRegressor(**final_lgb_params)

model.fit(
    X_train, y_train,
    sample_weight=w_train,
    eval_set=[(X_test, y_test)],
    eval_sample_weight=[w_test],
    eval_metric='poisson',
    callbacks=[lgb.early_stopping(150, verbose=100)],
    categorical_feature=categorical_features
)

print("Model training complete.")

# Generate predictions
print("Generating predictions...")
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)
pred_oot = model.predict(X_oot)

print("Predictions complete. You can now run your decile analysis.")

[I 2025-11-13 04:53:12,418] A new study created in memory with name: no-name-a0e41b4b-b723-460a-b2c1-d892d876d864


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-11-13 04:53:12,960] Trial 0 finished with value: 0.6968784975536787 and parameters: {'learning_rate': 0.06340492055590749, 'num_leaves': 59, 'max_depth': 5, 'min_child_weight': 135, 'subsample': 0.9122653708617809, 'colsample_bytree': 0.8328357661496912, 'reg_alpha': 0.010982464043268194, 'reg_lambda': 0.010240302903876439}. Best is trial 0 with value: 0.6968784975536787.
[I 2025-11-13 04:53:13,455] Trial 1 finished with value: 0.6984395693722747 and parameters: {'learning_rate': 0.05240122207346396, 'num_leaves': 49, 'max_depth': 7, 'min_child_weight': 53, 'subsample': 0.930452420808481, 'colsample_bytree': 0.9691841759698168, 'reg_alpha': 0.08457922176772015, 'reg_lambda': 0.25906375708427803}. Best is trial 0 with value: 0.6968784975536787.
[I 2025-11-13 04:53:14,171] Trial 2 finished with value: 0.6982017594599189 and parameters: {'learning_rate': 0.03146452267705702, 'num_leaves': 58, 'max_depth': 7, 'min_child_weight': 128, 'subsample': 0.8590929646416892, 'colsample_bytr

In [7]:
# 4. Decile Analysis Function
def create_decline_analysis(y_true, y_pred_count, exposure, set_name):
    print(f"\n Decile Analysis: {set_name} Set")
    
    eval_df = pd.DataFrame({
        'actual_count': y_true,
        'predicted_count': y_pred_count,
        'exposure': exposure
    })
    eval_df = eval_df[eval_df['exposure'] > 0].copy()

    overall_actual_ir = eval_df['actual_count'].sum() / eval_df['exposure'].sum()
    overall_pred_ir = eval_df['predicted_count'].sum() / eval_df['exposure'].sum()
    
    # Binning based on the raw predicted count
    eval_df['decile'] = pd.qcut(eval_df['predicted_count'], 10, labels=False, duplicates='drop')


    decile_groups = eval_df.groupby('decile')
    
    decile_summary = pd.DataFrame({
        'Policies': decile_groups.size(),
        'Total Exposure': decile_groups['exposure'].sum(),
        'Actual Claims': decile_groups['actual_count'].sum(),
        'Predicted IR': decile_groups['predicted_count'].mean() 
    })
    
    decile_summary['Actual IR'] = decile_summary['Actual Claims'] / decile_summary['Total Exposure']
    decile_summary['Delta'] = (decile_summary['Predicted IR'] / decile_summary['Actual IR']) - 1

    if overall_actual_ir > 0:
        decile_summary['Lift'] = decile_summary['Actual IR'] / overall_actual_ir
    else:
        decile_summary['Lift'] = np.nan
        
    top_decile_lift = np.nan
    top_to_bottom_ratio = np.nan
    
    if 9 in decile_summary.index:
        top_decile_lift = decile_summary.loc[9, 'Lift']
        top_decile_ir = decile_summary.loc[9, 'Actual IR']
        if 0 in decile_summary.index:
            bottom_decile_ir = decile_summary.loc[0, 'Actual IR']
            if bottom_decile_ir > 0:
                top_to_bottom_ratio = top_decile_ir / bottom_decile_ir


    format_cols_pct = ['Actual IR', 'Predicted IR', 'Delta']
    for col in format_cols_pct:
        decile_summary[col] = (decile_summary[col] * 100).map('{:,.2f}%'.format)

    decile_summary['Lift'] = decile_summary['Lift'].map('{:,.2f}'.format)

    display_cols = ['Policies', 'Actual IR', 'Predicted IR', 'Delta']
    print(decile_summary[display_cols])
    
    print(f"\nOverall Actual IR:    {overall_actual_ir:.4%}")
    print(f"Overall Predicted IR: {overall_pred_ir:.4%}")
    
    print(f"Lift:  {top_to_bottom_ratio:.2f}x")
    

# 5. Run Analysis & Report
create_decline_analysis(y_train, pred_train, w_train, "Train")
create_decline_analysis(y_test, pred_test, w_test, "Test")
create_decline_analysis(y_oot, pred_oot, w_oot, "OOT")


 Decile Analysis: Train Set
        Policies Actual IR Predicted IR    Delta
decile                                          
0           5004    18.05%       21.71%   20.32%
1           5005    25.54%       28.72%   12.45%
2           5001    30.24%       32.07%    6.06%
3           5003    33.46%       34.93%    4.40%
4           5003    37.03%       37.71%    1.84%
5           5003    40.59%       40.73%    0.34%
6           5003    42.94%       44.14%    2.80%
7           5003    49.21%       48.61%   -1.23%
8           5003    57.98%       54.78%   -5.51%
9           5003    78.46%       70.59%  -10.04%

Overall Actual IR:    41.3488%
Overall Predicted IR: 41.3493%
Lift:  4.35x

 Decile Analysis: Test Set
        Policies Actual IR Predicted IR   Delta
decile                                         
0            862    22.30%       20.35%  -8.74%
1            861    27.01%       27.32%   1.13%
2            862    32.24%       30.95%  -4.00%
3            861    32.01%       33.83%

In [ ]:
# 6. Post-Model Analysis
def create_feature_level_analysis(X_data, y_true, y_pred_count, exposure, model, set_name):

    print(f"\n Feature-Level Analysis: {set_name} Set")
    
    eval_df = X_data.copy()
    eval_df['actual_count'] = y_true
    eval_df['predicted_count'] = y_pred_count
    eval_df['exposure'] = exposure
    
    eval_df = eval_df[eval_df['exposure'] > 0].copy()
        
    try:
        top_features = model.booster_.feature_name()[:25]
        print(f"Analyzing features: {top_features}")
    except Exception as e:
        print(f"Couldn't get feature names, skipping: {e}")
        return

    for col in top_features:
        if col not in eval_df.columns:
            print(f"Skipping '{col}', not in X_data.")
            continue
            
        print(f"\nAnalyzing Feature: '{col}'")
        
        feature_data = eval_df[col]
        if feature_data.isnull().any():
            if feature_data.dtype.name == 'category':
                feature_data = feature_data.cat.add_categories('NaN').fillna('NaN')
            else:
                feature_data = feature_data.fillna('NaN')
                
        groups = eval_df.groupby(feature_data)
        
        summary_df = pd.DataFrame({
            'Policies': groups.size(),
            'Total Exposure': groups['exposure'].sum(),
            'Actual Claims': groups['actual_count'].sum(),
            'Predicted Claims': groups['predicted_count'].sum()
        })
        
        summary_df['Actual IR'] = summary_df['Actual Claims'] / summary_df['Total Exposure']
        summary_df['Predicted IR'] = summary_df['Predicted Claims'] / summary_df['Total Exposure']
        summary_df['Delta'] = summary_df['Actual IR'] - summary_df['Predicted IR']
        
        summary_df = summary_df.sort_values(by='Policies', ascending=False)
        
        format_cols_pct = ['Actual IR', 'Predicted IR', 'Delta']
        for pct_col in format_cols_pct:
            summary_df[pct_col] = (summary_df[pct_col] * 100).map('{:,.2f}%'.format)

        with pd.option_context('display.max_rows', 25):
             print(summary_df[['Policies', 'Actual IR', 'Predicted IR', 'Delta']].head(25))
        
        if len(summary_df) > 20:
            print(f"  ... (and {len(summary_df) - 20} more values)")

# Run analysis on OOT set
print("\n Feature-Level Analysis on OOT Set...")
create_feature_level_analysis(
    X_oot, 
    y_oot, 
    pred_oot, 
    w_oot, 
    model, 
    "OOT"
)

In [8]:
import pandas as pd

def get_summary_stats(y_true, y_pred_count, exposure, set_name):
    """Calculates summary stats for a given dataset."""
    
    # Filter out records with no exposure, matching your decile function
    mask = exposure > 0
    y_true_filtered = y_true[mask]
    y_pred_count_filtered = y_pred_count[mask]
    exposure_filtered = exposure[mask]

    # Calculate total sums
    total_exposure = exposure_filtered.sum()
    actual_claims = y_true_filtered.sum()
    predicted_claims = y_pred_count_filtered.sum() 
    
    # Calculate overall rates
    actual_ir = (actual_claims / total_exposure)
    predicted_ir = (predicted_claims / total_exposure)
    
    return {
        'Set': set_name,
        'Total Exposure': total_exposure,
        'Actual Claims': actual_claims,
        'Predicted Claims': predicted_claims,
        'Actual IR': actual_ir,
        'Predicted IR': predicted_ir
    }

# --- Run Calculations ---
# Assumes y_train, pred_train, w_train, etc., are in memory
summary_list = []
summary_list.append(get_summary_stats(y_train, pred_train, w_train, "Train"))
summary_list.append(get_summary_stats(y_test, pred_test, w_test, "Test"))
summary_list.append(get_summary_stats(y_oot, pred_oot, w_oot, "OOT"))

# --- Create and Display the DataFrame ---
summary_df = pd.DataFrame(summary_list).set_index('Set')

# Define formatting
formatters = {
    'Total Exposure': '{:,.2f}'.format,
    'Actual Claims': '{:,.0f}'.format,
    'Predicted Claims': '{:,.2f}'.format,
    'Actual IR': '{:.4%}'.format,
    'Predicted IR': '{:.4%}'.format
}

print("--- Model Performance Summary ---")
# Use to_string() to ensure all formatters apply
print(summary_df.to_string(formatters=formatters))

--- Model Performance Summary ---
      Total Exposure Actual Claims Predicted Claims Actual IR Predicted IR
Set                                                                       
Train      50,090.90        20,712        20,712.25  41.3488%     41.3493%
Test        7,661.04         2,877         3,455.82  37.5536%     45.1090%
OOT         3,617.49         1,402         2,393.08  38.7561%     66.1530%
